In [1]:
import os
currentDirectory = os.getcwd()
data_dir = currentDirectory[:-7] + '/input/'
print(data_dir)

from xmitgcm import open_mdsdataset
ds1 = open_mdsdataset(data_dir, geometry='cartesian', endian='<',prefix=['statevars'],iters=range(120*1860,170*1860,1860))

/scratch/jxchang/HighRes1/results/TideU008N0SimH200ho140Ah0200Cdqdt003/input/


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import matplotlib.colors as colors
import matplotlib.animation as animation
import math
import xarray as xr
import xgcm
import pandas as pd
%matplotlib inline

In [3]:
t = 0
h0 = 140.
viscAh=2.e-2
rhoNil=999.8
g = 9.8
u0=0.08
om=2*np.pi/12.4/3600 


xmin = -10
xmax = 10
vmax = 3
vmin = -3

numcolt=21
numcolv=30

ttlen=len(ds1.time)
print('the length of time:' + str(ttlen) )

the length of time:50


In [4]:
xg=ds1.coords["XG"]
xc=ds1.coords["XC"]
yg=ds1.coords["YG"]
yc=ds1.coords["YC"]
z=ds1.coords["Z"]
time=ds1.coords['time'].values/np.timedelta64(1, 's')/3600
yc0 = (ds1.YC/1000.-ds1.YC.mean()/1000.)
xc0 = (ds1.XC/1000.-ds1.XC.mean()/1000.)
grid = xgcm.Grid(ds1, periodic=False)
print(grid)

<xgcm.Grid>
X Axis (not periodic, boundary=None):
  * center   XC --> left
  * left     XG --> center
Z Axis (not periodic, boundary=None):
  * center   Z --> left
  * left     Zl --> center
  * outer    Zp1 --> center
  * right    Zu --> center
Y Axis (not periodic, boundary=None):
  * center   YC --> left
  * left     YG --> center
T Axis (not periodic, boundary=None):
  * center   time


In [5]:
time_ns = pd.timedelta_range(start='62 H', periods=ttlen,freq='1860S')
print(time_ns)

TimedeltaIndex(['2 days 14:00:00', '2 days 14:31:00', '2 days 15:02:00',
                '2 days 15:33:00', '2 days 16:04:00', '2 days 16:35:00',
                '2 days 17:06:00', '2 days 17:37:00', '2 days 18:08:00',
                '2 days 18:39:00', '2 days 19:10:00', '2 days 19:41:00',
                '2 days 20:12:00', '2 days 20:43:00', '2 days 21:14:00',
                '2 days 21:45:00', '2 days 22:16:00', '2 days 22:47:00',
                '2 days 23:18:00', '2 days 23:49:00', '3 days 00:20:00',
                '3 days 00:51:00', '3 days 01:22:00', '3 days 01:53:00',
                '3 days 02:24:00', '3 days 02:55:00', '3 days 03:26:00',
                '3 days 03:57:00', '3 days 04:28:00', '3 days 04:59:00',
                '3 days 05:30:00', '3 days 06:01:00', '3 days 06:32:00',
                '3 days 07:03:00', '3 days 07:34:00', '3 days 08:05:00',
                '3 days 08:36:00', '3 days 09:07:00', '3 days 09:38:00',
                '3 days 10:09:00', '3 days 10:40:00

In [6]:
print(ds1.coords['time'])

<xarray.DataArray 'time' (time: 50)>
array([223200000000000, 225060000000000, 226920000000000, 228780000000000,
       230640000000000, 232500000000000, 234360000000000, 236220000000000,
       238080000000000, 239940000000000, 241800000000000, 243660000000000,
       245520000000000, 247380000000000, 249240000000000, 251100000000000,
       252960000000000, 254820000000000, 256680000000000, 258540000000000,
       260400000000000, 262260000000000, 264120000000000, 265980000000000,
       267840000000000, 269700000000000, 271560000000000, 273420000000000,
       275280000000000, 277140000000000, 279000000000000, 280860000000000,
       282720000000000, 284580000000000, 286440000000000, 288300000000000,
       290160000000000, 292020000000000, 293880000000000, 295740000000000,
       297600000000000, 299460000000000, 301320000000000, 303180000000000,
       305040000000000, 306900000000000, 308760000000000, 310620000000000,
       312480000000000, 314340000000000], dtype='timedelta64[ns

In [7]:
print(time)

[62.         62.51666667 63.03333333 63.55       64.06666667 64.58333333
 65.1        65.61666667 66.13333333 66.65       67.16666667 67.68333333
 68.2        68.71666667 69.23333333 69.75       70.26666667 70.78333333
 71.3        71.81666667 72.33333333 72.85       73.36666667 73.88333333
 74.4        74.91666667 75.43333333 75.95       76.46666667 76.98333333
 77.5        78.01666667 78.53333333 79.05       79.56666667 80.08333333
 80.6        81.11666667 81.63333333 82.15       82.66666667 83.18333333
 83.7        84.21666667 84.73333333 85.25       85.76666667 86.28333333
 86.8        87.31666667]


In [8]:
# depth mean velocity
U0W=((ds1['UVEL']*ds1['drF']*ds1['hFacW']).sum('Z'))/(ds1.drF*ds1.hFacW*ds1.maskW).sum('Z')
V0S=((ds1['VVEL']*ds1['drF']*ds1['hFacS']).sum('Z'))/(ds1.drF*ds1.hFacS*ds1.maskS).sum('Z')
print(U0W[4,57,650].values)
ds1['U0W']=xr.DataArray(U0W.data,coords=[time_ns,yc,xg],dims=['time','YC','XG'])
ds1['V0S']=xr.DataArray(V0S.data,coords=[time_ns,yg,xc],dims=['time','YG','XC'])
print(ds1.U0W[4,57,650].values)
#del U0W,V0S

0.07303324871228556
0.07303324871228556


/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


In [9]:
# baroclinic velocity
ds1['maskL']=grid.interp(ds1.hFacC,'Z',to='left', boundary='extrapolate')  #mask for wvel
ds1['upW']=(ds1['UVEL']-ds1['U0W'])*ds1['maskW']
ds1['vpS']=(ds1['VVEL']-ds1['V0S'])*ds1['maskS']
wp=ds1['WVEL'].where(ds1['maskL'] !=0, np.nan)

In [10]:
# horizontal dissipation
ds1['maskZ'] = grid.interp(ds1.hFacS, 'X', boundary='extrapolate') #for dvdx
ds1['hFacZ'] = grid.interp(ds1.hFacS, 'X', boundary='extrapolate') #for dvdx
ds1['hFacWL'] = grid.interp(ds1.hFacW, 'Z', to='left', boundary='extrapolate')  #for dwdx
print(ds1.hFacWL.dims)
ds1['hFacSL'] = grid.interp(ds1.hFacS, 'Z', to='left', boundary='extrapolate')  #for dwdy
print(ds1.hFacSL.dims)
ds1['drL']=grid.interp(ds1.drF, 'Z', to='left', boundary='extrapolate')

('Zl', 'YC', 'XG')
('Zl', 'YG', 'XC')


In [11]:
print(((ds1['UVEL']*ds1['drF']*ds1['hFacW']).sum('Z'))[4,57,650].values)

14.606649742457112


In [12]:
print(((ds1.drF*ds1.hFacW*ds1.maskW).sum('Z'))[57,650].values)

200.0


In [13]:
print(ds1.UVEL.isel(time=4,Z=6,YC=57,XG=650).values)
print(ds1.VVEL[4,6,57,650].values)

0.1499497126082144
-0.07057070426832507


In [14]:
print(U0W.data[4,57,650])

dask.array<getitem, shape=(), dtype=float64, chunksize=(), chunktype=numpy.ndarray>


In [15]:
print(ds1.U0W.isel(time=4,YC=57,XG=650).values)

0.07303324871228556


/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


In [16]:
print(U0W.isel(time=4,YC=57,XG=650).values)

0.07303324871228556


In [17]:
print(ds1.U0W)

<xarray.DataArray 'U0W' (time: 50, YC: 120, XG: 1440)>
dask.array<truediv, shape=(50, 120, 1440), dtype=float64, chunksize=(1, 120, 1440), chunktype=numpy.ndarray>
Coordinates:
  * YC       (YC) float64 12.5 37.5 62.5 87.5 ... 2.938e+03 2.962e+03 2.988e+03
  * XG       (XG) float64 3.411e-13 713.5 1.407e+03 ... 7.859e+04 7.929e+04
    dyG      (YC, XG) float64 dask.array<chunksize=(120, 1440), meta=np.ndarray>
    dxC      (YC, XG) float64 dask.array<chunksize=(120, 1440), meta=np.ndarray>
    rAw      (YC, XG) float64 dask.array<chunksize=(120, 1440), meta=np.ndarray>
    maskInW  (YC, XG) bool dask.array<chunksize=(120, 1440), meta=np.ndarray>
    iter     (time) int64 dask.array<chunksize=(1,), meta=np.ndarray>
  * time     (time) timedelta64[ns] 2 days 14:00:00 ... 3 days 15:19:00


In [18]:
(((ds1['UVEL']*ds1['drF']*ds1['hFacW']).sum('Z'))[4,57,650]/(ds1.drF*ds1.hFacW*ds1.maskW).sum('Z')[57,650]).values

array(0.07303325)

In [19]:
print(ds1.upW[4,6,57,650].values)
print(ds1.vpS[4,6,57,650].values)

/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


0.07691646389592885
-0.07137582220801968


In [20]:
#(xc,yc)
hDispbc1 = rhoNil*viscAh*(((grid.diff(ds1.upW* ds1.dyG , 'X', boundary='extrapolate')/ds1.rA)**2+(grid.diff(ds1.vpS* ds1.dxG , 'Y', boundary='extrapolate')/ds1.rA)**2)*(ds1['drF']*ds1['hFacC']))  #dudx & dvdy
#(xg,yg)
hDispbc2 = rhoNil*viscAh*(((grid.diff(ds1.vpS* ds1.dyC , 'X', boundary='extrapolate')/ds1.rAz)**2+(grid.diff(ds1.upW* ds1.dxC , 'Y', boundary='extrapolate')/ds1.rAz)**2)*(ds1['drF']*ds1['hFacZ'])) #dvdx & dudy
#(xg,yc)
hDispbc31 = rhoNil*viscAh*(((grid.diff(wp ,'X',boundary='extrapolate')/ds1.dxC)**2)*(ds1['drL']*ds1['hFacWL']))  #dwdx
#(xc,yg)
hDispbc32 = rhoNil*viscAh*(((grid.diff(wp ,'Y',boundary='extrapolate')/ds1.dyC)**2)*(ds1['drL']*ds1['hFacSL']))  #dwdy

In [21]:
print(hDispbc1[4,6,57,650].values)
print(hDispbc2[4,6,57,650].values)
print(hDispbc31[4,6,57,650].values)
print(hDispbc32[4,6,57,650].values)

/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


0.00021372819176079168
0.0001848673036410335
3.804114615921156e-07
7.294493963359113e-07


In [22]:
hDispbc = hDispbc1+grid.interp(hDispbc2,['X','Y'],boundary='extrapolate') \
         +grid.interp(hDispbc31,['X','Z'],boundary='extrapolate') \
         +grid.interp(hDispbc32,['Y','Z'],boundary='extrapolate')
print(hDispbc)

<xarray.DataArray (time: 50, Z: 40, YC: 120, XC: 1440)>
dask.array<add, shape=(50, 40, 120, 1440), dtype=float64, chunksize=(1, 38, 118, 1438), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) timedelta64[ns] 2 days 14:00:00 ... 3 days 15:19:00
  * Z        (Z) float64 -2.5 -7.5 -12.5 -17.5 ... -182.5 -187.5 -192.5 -197.5
  * YC       (YC) float64 12.5 37.5 62.5 87.5 ... 2.938e+03 2.962e+03 2.988e+03
  * XC       (XC) float64 356.8 1.06e+03 1.745e+03 ... 7.894e+04 7.964e+04
    rA       (YC, XC) float64 dask.array<chunksize=(120, 1440), meta=np.ndarray>
    Depth    (YC, XC) float64 dask.array<chunksize=(120, 1440), meta=np.ndarray>
    maskInC  (YC, XC) bool dask.array<chunksize=(120, 1440), meta=np.ndarray>
    drF      (Z) float64 dask.array<chunksize=(40,), meta=np.ndarray>
    PHrefC   (Z) float64 dask.array<chunksize=(40,), meta=np.ndarray>
    rhoRef   (Z) float64 dask.array<chunksize=(40,), meta=np.ndarray>
    hFacC    (Z, YC, XC) float64 dask.array<chunksize=(40, 120

In [23]:
fig = plt.figure(figsize=(11,8))
grid = plt.GridSpec(2, 2, height_ratios=[0.6,2.5], width_ratios=[1,0.1],wspace=0.02,hspace=0.38)

<Figure size 792x576 with 0 Axes>

In [24]:
SMALL_SIZE = 17
MEDIUM_SIZE = 19
BIGGER_SIZE = 21

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [25]:
#plot bathymetry
ax1=fig.add_subplot(grid[0,:-1])
ax1.plot(time,np.zeros(ttlen),color='gray',linestyle='--')
ax1.plot(time,ds1.UVEL.isel(Z=0,YC=0,XG=0),label='U(t,0,0,0)')
ax1.plot(time,u0*np.cos(om*ds1.time.values/np.timedelta64(1, 's')),label='Uinput')
ax1.set_xlim(min(time), max(time))
ax1.set_ylim(-u0-0.05, u0+0.05)
point, = ax1.plot([time[0]],[ds1.UVEL.isel(time=t,YC=40,XG=0,Z=0).data], 'go', color='red',ms=10)
ax1.set_xlabel('time [hr]')
ax1.set_ylabel('U [m/s]')
ax1.set_title('Uinput vs U[t,0,0,0] ', loc='left')



Text(0.0, 1.0, 'Uinput vs U[t,0,0,0] ')

In [26]:
dmax=10**(-1)
dmin=10**(-6)

# plot epsh
ax2=fig.add_subplot(grid[1,0])
cmap = cm.Spectral_r
#levels = np.linspace(23, 45, num=numcolt)
#norm = BoundaryNorm(levels, ncolors=cmap.N, clip=True)
#opts = {'cmap':cm.get_cmap(cmap, len(levels) - 1), 'norm': norm}

pcm=ax2.pcolormesh(xc0,yc0, hDispbc.isel(time=t,Z=6), vmax=dmax, vmin=dmin
                ,norm=colors.LogNorm(vmin=dmin, vmax=dmax),cmap='RdYlBu_r',rasterized=True)
#cset1 = ax2.contourf(ds1.XC0,  hDispbc.isel(time=t,Z=6))

cax = fig.add_subplot(grid[1,-1])
cax.set_visible(False)
fig.colorbar(pcm, ax=cax,extend='both')
#fig.colorbar(cset1, ax=axs,ticks=np.arange(tmin, tmax+0.1, 5))
ax2.set_xlabel('X [km]')
ax2.set_ylabel('Y [km]')
ax2.set_title(r'$\epsilon_h$', loc='left')
ax2.set(xlim=(xmin, xmax))
#ax2.plot(ds1['XC0'],-ds1.Depth.isel(YC=60),lw=1.5)

/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  if sys.path[0] == '':
/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  if sys.path[0] == '':


[(-10.0, 10.0)]

In [27]:
fig
print(ax2.collections)
print(np.size(ax2.collections))

1


In [28]:
tt: int=0
def updateData(tt):

    print(tt)
    updatetime = time[tt]
    pcm.set_array(hDispbc[tt,6,:-1, :-1].values.flatten())
    point.set_data(updatetime,ds1.UVEL.isel(time=tt,YC=60,XG=0,Z=0))
    ax1.set_title('t= %2.3fhr' % updatetime)
    fig1 = plt.gcf()
    fig.savefig('./figsMag2/summary_epsh_z6_x10y_t%s.png' %(tt+120))



simulation = animation.FuncAnimation(fig, updateData, blit=False, frames= 40 , interval=40, repeat=False)
plt.draw()
plt.show()
simulation.save(filename='summary_epsh_z6_x10y.gif', writer='imagemagick', fps=2)

<Figure size 432x288 with 0 Axes>

0


/home/jxchang/jupyter_py3.7/lib/python3.7/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


<Figure size 432x288 with 0 Axes>

In [29]:
from IPython.display import Image
from IPython.display import display
display(Image(url='./summary_epsh_z6_x10y.gif'))